In [134]:
from cryptoadvance.specter.rpc import BitcoinCLI, RpcError
import numpy as np
#cli = BitcoinCLI("bitcoin", "secret", "localhost", "18443")
cli = BitcoinCLI("bitcoin", "dqptmms53t11i8vx93sp", "thundroid.local", "8332")
cli.getblockchaininfo()
# To investigate the Bitcoin API, here are some great resources:
# https://bitcoin.org/en/developer-reference#bitcoin-core-apis
# https://chainquery.com/bitcoin-cli
# https://github.com/ChristopherA/Learning-Bitcoin-from-the-Command-Line

{'chain': 'main',
 'blocks': 623012,
 'headers': 623012,
 'bestblockhash': '000000000000000000073ab61877ffe4f4044d6344854268999b1c01954ec7fc',
 'difficulty': 13912524048945.91,
 'mediantime': 1585221456,
 'verificationprogress': 0.9999962824704106,
 'initialblockdownload': False,
 'chainwork': '00000000000000000000000000000000000000000dec58e285b8018625b853dc',
 'size_on_disk': 305527731289,
 'pruned': False,
 'softforks': [{'id': 'bip34', 'version': 2, 'reject': {'status': True}},
  {'id': 'bip66', 'version': 3, 'reject': {'status': True}},
  {'id': 'bip65', 'version': 4, 'reject': {'status': True}}],
 'bip9_softforks': {'csv': {'status': 'active',
   'startTime': 1462060800,
   'timeout': 1493596800,
   'since': 419328},
  'segwit': {'status': 'active',
   'startTime': 1479168000,
   'timeout': 1510704000,
   'since': 481824}},
 'warnings': ''}

Let's look into the result of the getblock-call. It needs a hash as param and we're using the getblockhash-funtion to obtain the hash via the desired blockheight.

In [135]:
print(cli.getblock(cli.getblockhash(2)).keys())
print(list(cli.getblock(cli.getblockhash(1)).values()))

dict_keys(['hash', 'confirmations', 'strippedsize', 'size', 'weight', 'height', 'version', 'versionHex', 'merkleroot', 'tx', 'time', 'mediantime', 'nonce', 'bits', 'difficulty', 'chainwork', 'nTx', 'previousblockhash', 'nextblockhash'])
['00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 623012, 215, 215, 860, 1, 1, '00000001', '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098', ['0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098'], 1231469665, 1231469665, 2573394689, '1d00ffff', 1, '0000000000000000000000000000000000000000000000000000000200020002', 1, '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f', '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd']


We will need a way to calculate the subsidy. Here is a function which does that:

In [201]:
def blocksubsidy(height):
    ' see https://gist.github.com/nelruk/d00d785f84e47288b0ab734d4aab5f49 '
    period = int(height / 210000)
    subsidy=50
    for i in range(0,period):
        subsidy = subsidy /2
    return subsidy


We will need to iterate through all the blocks. As the hash of the next block is part of the json, we don't have to use the geblockhash-call. This will make it faster. So here is a simplified version of how it looks like to navigate through the first 1000 blocks. We will also sum up the stock and measure the time this will take:

In [213]:
import time
import csv
from datetime import datetime
#blocks = cli.getblockchaininfo()["blocks"]
# Lets' only iterate 1000 blocks
blocks = 1000
tic = time.perf_counter()
hash = cli.getblockhash(1)
stock = 0
for i in range(0, 1000):
    block = cli.getblock(hash)
    stock = stock + blocksubsidy(i)
    hash = block['nextblockhash']

toc = time.perf_counter()
print(f"created the array in {toc - tic:0.4f} seconds")
print(f"In {blocks} blocks, btc {stock} has been created")

created the array in 3.6843 seconds
In 1000 blocks, btc 50000 has been created


So now we'll write out the stock for each day. For that, we'll check for each block, whether i has been created on a different day than the former block.

In [225]:
import time
import csv
from datetime import datetime
blocks = cli.getblockchaininfo()["blocks"]
# ommitting block 0 as it's non standard (18 elements instead of 19 as all the others)
tic = time.perf_counter()
hash = cli.getblockhash(1)
stock = 0
former_block_day = datetime.utcfromtimestamp(cli.getblock(hash)["time"]).strftime('%Y-%m-%d')
with open('days.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(0, blocks):
        block = cli.getblock(hash)
        timestamp = block['time']
        block_day = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')
        if (block_day != former_block_day):
            writer.writerow([block_day,i,stock])
            print(block_day)
        stock = stock + blocksubsidy(i)
        former_block_day = block_day
        hash = block['nextblockhash']
    
toc = time.perf_counter()
print(f"created the csv in {toc - tic:0.4f} seconds")

2009-01-10
2009-01-11
2009-01-12
2009-01-13
2009-01-14
2009-01-15
2009-01-16
2009-01-17
2009-01-18
2009-01-19
2009-01-20
2009-01-21
2009-01-22
2009-01-23
2009-01-24
2009-01-25
2009-01-26
2009-01-27
2009-01-28
2009-01-29
2009-01-30
2009-01-31
2009-02-01
2009-02-02
2009-02-03
2009-02-04
2009-02-05
2009-02-06
2009-02-07
2009-02-08
2009-02-09
2009-02-10
2009-02-11
2009-02-12
2009-02-13
2009-02-14
2009-02-15
2009-02-16
2009-02-17
2009-02-18
2009-02-19
2009-02-20
2009-02-21
2009-02-22
2009-02-23
2009-02-24
2009-02-25
2009-02-26
2009-02-27
2009-02-28
2009-03-01
2009-03-02
2009-03-03
2009-03-04
2009-03-05
2009-03-06
2009-03-07
2009-03-08
2009-03-09
2009-03-10
2009-03-11
2009-03-12
2009-03-13
2009-03-14
2009-03-15
2009-03-16
2009-03-17
2009-03-18
2009-03-19
2009-03-20
2009-03-21
2009-03-22
2009-03-23
2009-03-24
2009-03-25
2009-03-26
2009-03-27
2009-03-28
2009-03-29
2009-03-30
2009-03-31
2009-04-01
2009-04-02
2009-04-03
2009-04-04
2009-04-05
2009-04-06
2009-04-07
2009-04-08
2009-04-09
2009-04-10

2011-01-09
2011-01-10
2011-01-11
2011-01-12
2011-01-13
2011-01-14
2011-01-15
2011-01-16
2011-01-17
2011-01-18
2011-01-19
2011-01-20
2011-01-21
2011-01-22
2011-01-23
2011-01-24
2011-01-25
2011-01-26
2011-01-27
2011-01-28
2011-01-29
2011-01-30
2011-01-31
2011-02-01
2011-02-02
2011-02-03
2011-02-04
2011-02-05
2011-02-06
2011-02-07
2011-02-08
2011-02-09
2011-02-10
2011-02-11
2011-02-12
2011-02-13
2011-02-14
2011-02-15
2011-02-16
2011-02-17
2011-02-18
2011-02-19
2011-02-20
2011-02-21
2011-02-22
2011-02-23
2011-02-24
2011-02-25
2011-02-26
2011-02-27
2011-02-28
2011-03-01
2011-03-02
2011-03-03
2011-03-04
2011-03-05
2011-03-06
2011-03-07
2011-03-08
2011-03-09
2011-03-10
2011-03-11
2011-03-12
2011-03-13
2011-03-14
2011-03-15
2011-03-16
2011-03-17
2011-03-18
2011-03-19
2011-03-20
2011-03-21
2011-03-22
2011-03-23
2011-03-24
2011-03-25
2011-03-26
2011-03-27
2011-03-28
2011-03-29
2011-03-30
2011-03-31
2011-04-01
2011-04-02
2011-04-03
2011-04-04
2011-04-05
2011-04-06
2011-04-07
2011-04-08
2011-04-09

2013-01-09
2013-01-10
2013-01-11
2013-01-12
2013-01-13
2013-01-14
2013-01-15
2013-01-16
2013-01-17
2013-01-18
2013-01-19
2013-01-20
2013-01-21
2013-01-22
2013-01-23
2013-01-24
2013-01-25
2013-01-26
2013-01-27
2013-01-28
2013-01-29
2013-01-30
2013-01-31
2013-02-01
2013-02-02
2013-02-03
2013-02-04
2013-02-05
2013-02-06
2013-02-07
2013-02-08
2013-02-09
2013-02-10
2013-02-11
2013-02-12
2013-02-13
2013-02-14
2013-02-15
2013-02-16
2013-02-17
2013-02-18
2013-02-19
2013-02-20
2013-02-21
2013-02-22
2013-02-23
2013-02-24
2013-02-25
2013-02-26
2013-02-27
2013-02-28
2013-03-01
2013-03-02
2013-03-03
2013-03-04
2013-03-05
2013-03-06
2013-03-07
2013-03-08
2013-03-09
2013-03-10
2013-03-11
2013-03-12
2013-03-13
2013-03-14
2013-03-15
2013-03-16
2013-03-17
2013-03-18
2013-03-19
2013-03-20
2013-03-21
2013-03-22
2013-03-23
2013-03-24
2013-03-25
2013-03-26
2013-03-27
2013-03-28
2013-03-29
2013-03-30
2013-03-31
2013-04-01
2013-04-02
2013-04-03
2013-04-04
2013-04-05
2013-04-06
2013-04-07
2013-04-08
2013-04-09

2014-12-23
2014-12-24
2014-12-25
2014-12-26
2014-12-27
2014-12-28
2014-12-29
2014-12-30
2014-12-31
2015-01-01
2015-01-02
2015-01-03
2015-01-04
2015-01-05
2015-01-06
2015-01-07
2015-01-08
2015-01-09
2015-01-10
2015-01-11
2015-01-12
2015-01-13
2015-01-14
2015-01-15
2015-01-16
2015-01-17
2015-01-18
2015-01-19
2015-01-20
2015-01-21
2015-01-22
2015-01-23
2015-01-24
2015-01-25
2015-01-26
2015-01-27
2015-01-28
2015-01-29
2015-01-30
2015-01-31
2015-02-01
2015-02-02
2015-02-03
2015-02-04
2015-02-05
2015-02-06
2015-02-07
2015-02-08
2015-02-09
2015-02-10
2015-02-11
2015-02-12
2015-02-13
2015-02-14
2015-02-15
2015-02-16
2015-02-17
2015-02-18
2015-02-19
2015-02-20
2015-02-21
2015-02-22
2015-02-23
2015-02-24
2015-02-25
2015-02-26
2015-02-27
2015-02-28
2015-03-01
2015-03-02
2015-03-03
2015-03-04
2015-03-05
2015-03-06
2015-03-07
2015-03-08
2015-03-09
2015-03-10
2015-03-11
2015-03-12
2015-03-13
2015-03-14
2015-03-15
2015-03-16
2015-03-17
2015-03-18
2015-03-19
2015-03-20
2015-03-21
2015-03-22
2015-03-23

2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-21
2017-01-22
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-04
2017-03-05
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-11
2017-03-12
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-18
2017-03-19
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-25
2017-03-26
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-01
2017-04-02
2017-04-03
2017-04-04

2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01
2019-04-02
2019-04-03
2019-04-04
2019-04-05
2019-04-06
2019-04-07
2019-04-08
2019-04-09
2019-04-10
2019-04-11
2019-04-12
2019-04-13
2019-04-14
2019-04-15
2019-04-16
2019-04-17
2019-04-18
2019-04-19

The first blocks are running quite fast. But the increasing number of transactions in later blocks will slow it down tremendously. 

| Number of block | time to create array   |
|------|------|
|   1000  |  4s |
|  10000  |  38s |
|  20000  |  511s |
| 600000  |  43351s (12h) |

So this is not very effective as on top of that, also the algorithm needs to be starting from scratch if you want to redo that.
So improving that would complicate the code tremendously, so won't do it for now.